# Find Tbreak and Yield Strength at Tbreak 
## In this function, it suppose to output the fitting plot for temperature and according Tbreak and yield strength at Tbreak

In [1]:


import numpy as np
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt



def calculate_tbreak_and_save_plot(temperatures, ys_observed,MeltTemp,comp_name):
    def calculate_mse_intercept(params):
        Tbreak, c1, b1, c2 = params
        b2 = b1 + (c1 - c2) * (Tbreak / Tm)  # Enforce intersection at Tbreak
        log_ys_model = np.where(
            temperatures <= Tbreak,
            c1 * (temperatures / Tm) + b1,
            c2 * (temperatures / Tm) + b2
        )
        log_ys_observed = np.log(ys_observed)
        mse = np.mean((log_ys_model - log_ys_observed) ** 2)
        return mse
    temperatures = np.array(temperatures)
    ys_observed = np.array(ys_observed)

    if ys_observed[-1]>10:
        temperatures=np.append(temperatures,MeltTemp*0.85)
        ys_observed=np.append(ys_observed,1)

    # Check and replace 0 yield strength values with 1 to avoid log issues
    ys_observed[ys_observed <= 0] = 1

    # Assume a melting temperature Tm (This value should be estimated or provided)
    Tm=temperatures[-1]

    # Use differential evolution to optimize the parameters with the enforced intersection
    paraBound=35
    bounds_intercept = [(temperatures.min(), temperatures.max()*0.8), (-paraBound, 0), (-paraBound, paraBound), (-paraBound, 0)]
    result_intercept = differential_evolution(calculate_mse_intercept, bounds_intercept)

    # Extract the optimized parameters
    Tbreak_opt_intercept, c1_opt_intercept, b1_opt_intercept, c2_opt_intercept = result_intercept.x

    # Calculate b2 with the enforced condition
    b2_opt_intercept = b1_opt_intercept + (c1_opt_intercept - c2_opt_intercept) * (Tbreak_opt_intercept / Tm)
    ys_at_tbreak = np.exp(c1_opt_intercept * (Tbreak_opt_intercept / Tm) + b1_opt_intercept)
    # Generate a smooth temperature range for plotting
    temperature_smooth = np.linspace(temperatures.min(), temperatures.max(), 500)

    # Generate the modeled log(YS) using the optimized parameters with enforced intersection
    log_ys_model_smooth_intercept = np.where(
        temperature_smooth <= Tbreak_opt_intercept,
        c1_opt_intercept * (temperature_smooth / Tm) + b1_opt_intercept,
        c2_opt_intercept * (temperature_smooth / Tm) + b2_opt_intercept
    )

    if Tbreak_opt_intercept > 873:
        T600=c1_opt_intercept * (873 / Tm) + b1_opt_intercept
    else:
        T600=c2_opt_intercept * (873 / Tm) + b2_opt_intercept

    if Tbreak_opt_intercept > 1073:
        T800=c1_opt_intercept * (1073 / Tm) + b1_opt_intercept
    else:
        T800=c2_opt_intercept * (1073 / Tm) + b2_opt_intercept

    #Plotting the observed vs modeled yield strength with enforced intersection
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(temperatures, ys_observed, 'o', label='Observed YS (MPa)', markersize=8)
    plt.plot(temperature_smooth, np.exp(log_ys_model_smooth_intercept), '-', label='Modeled YS (MPa)', linewidth=2, color='orange')
    plt.axvline(x=Tbreak_opt_intercept, color='r', linestyle='--', label=f'Break Temperature = {Tbreak_opt_intercept:.2f} K')
    plt.title('Observed vs Modeled Yield Strength')
    plt.xlabel('Temperature (K)')
    plt.ylabel('Yield Strength (MPa)')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(temperatures, np.log(ys_observed), 'o', label='Observed log(YS)', markersize=8)
    plt.plot(temperature_smooth, log_ys_model_smooth_intercept, '-', label='Modeled log(YS)', linewidth=2, color='orange')
    plt.axvline(x=Tbreak_opt_intercept, color='r', linestyle='--', label=f'Break Temperature = {Tbreak_opt_intercept:.2f} K')
    plt.title('Observed vs Modeled log(Yield Strength)')
    plt.xlabel('Temperature (K)')
    plt.ylabel('Log(Yield Strength)')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    
    plt.savefig(f'C:\\Users\\Sir\\OneDrive\\Univeristy\\HEA Analysis\\4K06\\Code\\Data Folder\\Processed 0922\\Line Fit\\{comp_name}_fitted_plot.png')
    plt.close()
    return Tbreak_opt_intercept, ys_at_tbreak,np.exp(T600),np.exp(T800)


In [2]:
# Process each composition and calculate Tbreak and YS at Tbreak
import pandas as pd

data=pd.read_csv(r'C:\Users\Sir\OneDrive\Univeristy\HEA Analysis\4K06\Code\Data Folder\Processed 0922\FeatureCal.csv')

results = []
grouped = data.groupby('Comp')
for comp_name, group in grouped:
    temperatures = group['Temp'].values + 273.15  # Convert to Kelvin
    ys_observed = group['Yield'].values
    TM=group['Avg Melting Pt'].values[0]
    Tbreak, ys_at_tbreak,T600,T800 = calculate_tbreak_and_save_plot(temperatures, ys_observed,TM,comp_name)
    additional_columns = group.iloc[0][[
        'Al', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Ta', 'Ti', 'V', 'Zr', 'Nb', 'Mo', 
        'Zn', 'W', 'Sn', 'Nd', 'Hf', 'Si', 'VEC', 'ElectDiff', 'Atomic Size Diff', 
        'Mixing Enthalpy', 'Mixing Entropy', 'AVG Shear Modulus', 'AVG Bulk Modulus', 
        'AVG Possion Ratio', 'Itinerate Electron', 'Cohesive Energy', 'Gamma', 
        'Local Size Mismatch', 'Local Shear Modulus Mismatch', 'Local Electronegativity Mismatch', 
        'Avg Melting Pt'
    ]].to_dict()

    results.append({
        'Comp': comp_name, 
        'Tbreak': Tbreak, 
        'YS at Tbreak': ys_at_tbreak, 
        'Ys at 600C':T600,
        'Ys at 800C':T800,
        **additional_columns  # Include the additional columns
    })
pd.DataFrame(results).to_csv(r"C:\Users\Sir\OneDrive\Univeristy\HEA Analysis\4K06\Code\Data Folder\Processed 0922\Tbreak_Yield.csv")

In [10]:
pd.DataFrame(results).to_csv(r"C:\Users\Sir\OneDrive\Univeristy\HEA Analysis\4K06\Code\Data Folder\Processed 0922\Tbreak_Yield.csv")